In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# %env

True

In [2]:
import praw

reddit = praw.Reddit(client_id = os.environ['CLIENT_ID'],
                     client_secret = os.environ['SECRET_KEY'],
                     username = os.environ["REDDIT_USERNAME"],
                     password = os.environ["USER_PASSWORD"],
                     user_agent = 'Windows:MyAPI/0.0.1 (by AdMost7569)')

at first, i downloaded posts and submissions of the given sub-reddit, but than i realized, that i can not download history
submission, so i neglected that path, and downloaded the data, as a dataset, via:
https://academictorrents.com/details/9a3f77cf1b16f064b8f82e75ee8d470b49c90512

## read zst downloaded files

### load submissions

In [1]:
import zstandard as zstd
import pandas as pd

submissions_file = r"C:\Users\yaron\projects\BGU - data mining\project\IsraelPalestine\IsraelPalestine_submissions\IsraelPalestine_submissions"

df = pd.read_json(submissions_file, lines=True)
df_posts = df[['name','title','selftext','author','created_utc','score','upvote_ratio','subreddit_id','num_comments']]
df_posts.columns = ["submission_id", "submission_title", "submission_text", "submission_author","submission_timestamp","submission_score","upvote_ration",
                                     "subreddit_id", "num_comments"]
# df_posts


In [9]:
df_posts.submission_id.nunique(), df_posts.shape

(7853, (7853, 9))

### load comments

In [62]:
comments_file = r"C:\Users\yaron\projects\BGU - data mining\project\IsraelPalestine\IsraelPalestine_comments\IsraelPalestine_comments"

df = pd.read_json(comments_file, lines=True)
list(df.columns)
df_comments = df[['body','author','created_utc','score','subreddit_id','name','parent_id','link_id']]
df_comments.columns = ["comment_text", "comment_author", "comment_timestamp", "comment_score",
                                      "subrredit_id", "comment_id", "to_comment",'submission_id']
df_comments.head(2)
df_comments_short = df_comments[:10000]
df_comments_short.head(2)

,comment_text,comment_author,comment_timestamp,comment_score,subrredit_id,comment_id,to_comment,submission_id
0,>But it's the third comment I write to you whi...,Islamist-Analyst,1696118627,1,t5_2sgz7,t1_k2xmnwc,t1_k2wt9rn,t3_16w3rfn
1,evening temporary been real quiet since this c...,cadmiumflowers,1696118690,1,t5_2sgz7,t1_k2xmsxd,t1_k2x2mba,t3_16vgm2t


In [101]:
df_posts.head(2)

True

In [105]:
df.shape

(325413, 74)

### create A table - first level comments  to posts

In [120]:
df_A = df_comments.merge(df_posts, left_on='to_comment', right_on='submission_id')
df_A.head(2)
df_A = df_A[['submission_id_x', 'submission_title', 'submission_text','submission_author',\
             'comment_id', 'comment_text','comment_author','comment_timestamp']]
df_A.columns = ['submission_id', 'submission_title', 'submission_text','submission_author',\
             'comment_id', 'comment_text','comment_author','comment_timestamp']

### create B table - comments to comments (post text is additional information)

In [109]:
df_B = pd.merge(df_comments, df_comments[['comment_id','comment_text','comment_author']],\
         left_on='to_comment', right_on='comment_id')
df_B = df_B.merge(df_posts[['submission_id', 'submission_title', 'submission_text']], on='submission_id')
df_B = df_B[['submission_id', 'submission_title', 'submission_text', 'comment_id_x','comment_text_x','comment_author_x',\
            'to_comment', 'comment_text_y','comment_author_y','comment_timestamp']] 

In [110]:
df_B.shape

(265369, 10)

In [125]:
def text_length(text):
    return len(text.split()) 

### pre-process - remove comments, that are too short...

In [145]:
mininmum_comment_size_in_words = 10

In [126]:
for text in ['submission_text', 'comment_text_x', 'comment_text_y']:
    name = 'len_' + text
    df_B[name] = df_B[text].apply(text_length)

In [144]:
df_B_filtered = df_B.loc[(df_B.len_submission_text>mininmum_comment_size_in_words) \
                         & (df_B.len_comment_text_x > mininmum_comment_size_in_words) \
                         & (df_B.len_comment_text_y > mininmum_comment_size_in_words)]

In [147]:
for text in ['submission_text', 'comment_text']:
    name = 'len_' + text
    df_A[name] = df_A[text].apply(text_length)

In [148]:
df_A_filtered = df_A.loc[(df_A.len_submission_text>mininmum_comment_size_in_words) \
                         & (df_A.len_comment_text > mininmum_comment_size_in_words)]

# work with data post llm - export data for llm processing

In [150]:
import pickle 
with open("df_A_filtered.pickle", 'wb') as f:
    pickle.dump(df_A_filtered, f)
    
with open("df_B_filtered.pickle", 'wb') as f:
    pickle.dump(df_B_filtered, f)
    
import pickle 
with open("df_posts.pickle", "wb") as f:
    pickle.dump(df_posts, f)

at this point, i moved to another notebook for working with the llm and embedding. at the end of that notebook, i re-loaded the results to this notebook and continued 

# import the llm answers and combine them to current data

In [298]:
import pickle 

with open("A_features.pickle", 'rb') as f:
    A_features = pickle.load(f)

with open("B_features.pickle", 'rb') as f:
    B_features = pickle.load(f)
    
with open("posts_features_from_posts_features.pickle", "rb") as f:
    posts_features_from_posts_features = pickle.load(f)
    
with open("df_A_filtered.pickle", 'rb') as f:
    df_A = pickle.load(f)

with open("df_B_filtered.pickle", 'rb') as f:
    df_B = pickle.load(f)
    
with open("df_posts.pickle", 'rb') as f:
    df_posts = pickle.load(f)
    


#### convert timestamp from unix to datetime

In [299]:
from datetime import datetime
df_A['comment_timestamp'] = df_A['comment_timestamp'].apply(datetime.fromtimestamp)
df_B['comment_timestamp'] = df_B['comment_timestamp'].apply(datetime.fromtimestamp)
df_posts['submission_timestamp'] = df_posts['submission_timestamp'].apply(datetime.fromtimestamp) 

### join antisemetism features to the text

#### posts

In [300]:
import pandas as pd
posts_features_df = pd.DataFrame(posts_features_from_posts_features, columns = ['submission_id', 'features'])
df_posts = df_posts.merge(posts_features_df, on='submission_id')
df_posts.head(1)

,submission_id,submission_title,submission_text,submission_author,submission_timestamp,submission_score,upvote_ration,subreddit_id,num_comments,vectors,features
0,t3_16wuocg,Need help gathering info regarding the conflict.,My school is conducting an MUN (Model United N...,Moon_Knight_69,2023-10-01 10:28:02,1,1.0,t5_2sgz7,53,"[-0.006348003633320332, -0.0027542905882000923...",{'promote or endorse harmful stereotypes about...


#### comments to posts (A)

In [301]:
df_A['features'] = A_features 
df_A.head(1)

,submission_id,submission_title,submission_text,submission_author,comment_id,comment_text,comment_author,comment_timestamp,len_submission_text,len_comment_text,features
1,t3_16wvusf,Neturei Karta did all the thinking for me,I was extremely brainwashed about 10 years ago...,Old_Calligrapher1563,t1_k2z6w0g,>I know most jews absolutely despise them but ...,pinchasthegris,2023-10-01 11:59:08,162,37,{'promote or endorse harmful stereotypes about...


#### comments to comments (B)

In [302]:
df_B.shape
B_features_df = pd.DataFrame(B_features, columns = ['index','comment_id_x', 'features'])
df_B = df_B.merge(B_features_df, on='comment_id_x')
df_B.head(1)

,submission_id,submission_title,submission_text,comment_id_x,comment_text_x,comment_author_x,to_comment,comment_text_y,comment_author_y,comment_timestamp,len_submission_text,len_comment_text_x,len_comment_text_y,index,features
0,t3_16wvusf,Neturei Karta did all the thinking for me,I was extremely brainwashed about 10 years ago...,t1_k2z76q7,I guess I'd use those two words somewhat inter...,Old_Calligrapher1563,t1_k2z6w0g,>I know most jews absolutely despise them but ...,pinchasthegris,2023-10-01 12:03:02,162,24,37,0,{'promote or endorse harmful stereotypes about...


### exclude adminstrators users

In [303]:
admin_users = ["IsraelPalestine-ModTeam", "AutoModerator","IsraelPalestine-Mods"]
df_A = df_A[~df_A['comment_author'].isin(admin_users)]
df_B = df_B[~df_B['comment_author_x'].isin(admin_users)]
df_posts = df_posts[~df_posts['submission_author'].isin(admin_users)]

### explode questions to columns

In [304]:
df_A.dropna(inplace=True)
for key in df_A['features'].iloc[0].keys():
    df_A[key] = df_A['features'].apply(lambda x: x[key])

df_B.dropna(inplace=True)
for key in df_B['features'].iloc[0].keys():
    df_B[key] = df_B['features'].apply(lambda x: x[key])
    
df_posts.dropna(inplace=True)
for key in df_posts['features'].iloc[0].keys():
    df_posts[key] = df_posts['features'].apply(lambda x: x[key])

### calc sum of antisemitism

In [305]:
df_posts['sum_of_antisemitism'] = df_posts.iloc[:,-10:].apply(lambda x: sum(x), axis=1)

df_A['sum_of_antisemitism'] = df_A.iloc[:,-10:].apply(lambda x: sum(x), axis=1)

df_B['sum_of_antisemitism'] = df_B.iloc[:,-10:].apply(lambda x: sum(x), axis=1)

### add embedding data 
calculated in the working with llm and embedding notebook

In [306]:
with open('vectors_Av3.pickle', 'rb') as f:
    vectors_A = pickle.load(f)
    
with open('vectors_Bv4.pickle', 'rb') as f:
    vectors_B = pickle.load(f)

In [307]:
vectors_A_df = pd.DataFrame(vectors_A, columns = ['comment_id', 'vectors'])
df_A = df_A.merge(vectors_A_df, on='comment_id')
df_A.shape

(42518, 23)

In [308]:
vectors_B_df = pd.DataFrame(vectors_B, columns = ['comment_id_x', 'vectors'])
df_B = df_B.merge(right = vectors_B_df, how='left', on='comment_id_x')
df_B.shape

(174157, 27)

### prepare a joint dataframe with all submission, comments. with embedding, dates and sum of anti

In [309]:
posts = df_posts[['submission_id','sum_of_antisemitism','submission_timestamp']]
posts.columns = ['id','sum_of_antisemitism','timestamp']

A = df_A[['comment_id','sum_of_antisemitism','comment_timestamp']]
A.columns = ['id','sum_of_antisemitism','timestamp']

B = df_B[['comment_id_x','sum_of_antisemitism','comment_timestamp']].dropna()
B.columns = ['id','sum_of_antisemitism','timestamp']

df_embbeded_stacked = pd.concat([posts,A,B])
print(df_embbeded_stacked.shape)

with open('df_all_texts_with_antisemitism_timestamps.pickle', 'wb') as f:
    pickle.dump(df_embbeded_stacked, f)

(224476, 3)


In [10]:
with open('df_all_texts_with_antisemitism_timestamps.pickle', 'rb') as f:
    df_embbeded_stacked = pickle.load(f)

# extract nodes ( as users)

In [310]:
submissioners = set(df_posts.submission_author.values)
commenters = set(df_A.comment_author.values).union(set(df_B.comment_author_x.values))
print(f'submissioners - {len(submissioners)} commenters {len(commenters)} union - {len(submissioners.union(commenters))}')

submissioners - 4603 commenters 20714 union - 22440


In [311]:
users = set(submissioners).union(commenters)

### define features for users to determine antisemetism charachter

In [313]:
import pandas as pd
import numpy as np
def calculate_num_of_posts(df_posts: pd.DataFrame,
                           df_A: pd.DataFrame,
                           df_B: pd.DataFrame,
                           column_posts: str,
                           column_A: str,
                           column_B: str,
                           user: str):

    num_of_posts = df_posts[df_posts[column_posts] == user].shape[0]
    num_of_comments = df_A[df_A[column_A] == user].shape[0] + df_B[df_B[column_B] == user].shape[0]

    return num_of_posts + num_of_comments


def calculate_num_antisemetic_comments(df_posts: pd.DataFrame,
                           df_A: pd.DataFrame,
                           df_B: pd.DataFrame,
                           column_posts: str,
                           column_A: str,
                           column_B: str,
                           user: str):
    num_of_antisemetic_posts = sum(df_posts[df_posts[column_posts] == user].sum_of_antisemitism > 0)
    num_of_antisemetic_comments = sum(df_A[df_A[column_A] == user].sum_of_antisemitism > 0) + \
                                  sum(df_B[df_B[column_B] == user].sum_of_antisemitism > 0)

    return num_of_antisemetic_posts + num_of_antisemetic_comments



def calculate_max_strength_antisemitism(df_posts: pd.DataFrame,
                           df_A: pd.DataFrame,
                           df_B: pd.DataFrame,
                           column_posts: str,
                           column_A: str,
                           column_B: str,
                           user: str):
    
    max_post = np.max(df_posts[df_posts[column_posts] == user].sum_of_antisemitism.values,initial = 0)
    max_A = np.max(df_A[df_A[column_A] == user].sum_of_antisemitism.values,initial = 0)
    max_B = np.max(df_B[df_B[column_B] == user].sum_of_antisemitism.values,initial = 0)
    
    return np.max([max_post,max_A,max_B])

def calculate_sum__antisemitism(df_posts: pd.DataFrame,
                           df_A: pd.DataFrame,
                           df_B: pd.DataFrame,
                           column_posts: str,
                           column_A: str,
                           column_B: str,
                           user: str):
    
    sum_post = np.sum(df_posts[df_posts[column_posts] == user].sum_of_antisemitism.values)
    sum_A = np.sum(df_A[df_A[column_A] == user].sum_of_antisemitism.values)
    sum_B = np.sum(df_B[df_B[column_B] == user].sum_of_antisemitism.values)
    
    return np.sum([sum_post,sum_A,sum_B])

### calculate features values for nodes (users)

In [315]:
df_users = pd.DataFrame(pd.Series(list(users)), columns=['users'])
df_users['num_of_posts_or_comments'] = df_users['users'].apply(lambda x: calculate_num_of_posts(df_posts,
                                                        df_A,
                                                        df_B,
                                                        'submission_author',
                                                        'comment_author',
                                                        'comment_author_x',
                                                         x))
df_users['num_antisemetic_comments'] = df_users['users'].apply(lambda x: calculate_num_antisemetic_comments(df_posts,
                                                        df_A,
                                                        df_B,
                                                        'submission_author',
                                                        'comment_author',
                                                        'comment_author_x',
                                                         x))
df_users['max_streanght_antisemitism'] = df_users['users'].apply(lambda x: calculate_max_strength_antisemitism(df_posts,
                                                        df_A,
                                                        df_B,
                                                        'submission_author',
                                                        'comment_author',
                                                        'comment_author_x',
                                                         x))

df_users['sum_antisemitism'] = df_users['users'].apply(lambda x: calculate_sum__antisemitism(df_posts,
                                                        df_A,
                                                        df_B,
                                                        'submission_author',
                                                        'comment_author',
                                                        'comment_author_x',
                                                         x))

df_users['mean_strength_antisemitism'] = df_users['sum_antisemitism'] / df_users['num_of_posts_or_comments']

### add timestamp aspect into users (first appear)

In [351]:
# import numpy as np

# # change column users to user_name
df_users.columns = ['user_name', 'num_of_posts_or_comments', 'num_antisemetic_comments',
       'max_streanght_antisemitism', 'sum_antisemitism',
       'mean_strength_antisemitism','timestamp']



posts = df_posts[['submission_author','submission_timestamp']]
posts.columns = ['user_name','timestamp']

A = df_A[['comment_author','comment_timestamp']]
A.columns = ['user_name','timestamp']

B = df_B[['comment_author_x','comment_timestamp']].dropna()
B.columns = ['user_name','timestamp']

df_users_stacked_with_timestamp = pd.concat([posts,A,B])
print(df_users_stacked_with_timestamp.shape)

df_users_stacked_with_timestamp = df_users_stacked_with_timestamp.sort_values(by='timestamp').groupby('user_name').first()
df_users = df_users.merge(df_users_stacked_with_timestamp, how='left', on='user_name')

df_users.head(2)

,user_name,num_of_posts_or_comments,num_antisemetic_comments,max_streanght_antisemitism,sum_antisemitism,mean_strength_antisemitism,timestamp
0,Inevitable_Drawer145,6,0,0,0,0.0,2023-10-15 04:12:25
1,Responsible_Ad6938,5,1,1,1,0.2,2023-10-23 17:00:20


In [356]:
import pickle
with open('df_users.pickle', 'wb') as f:
    pickle.dump(df_users, f)

In [5]:
import pickle
with open('df_users.pickle', 'rb') as f:
    df_users  = pickle.load(f)

In [317]:
df_users.sort_values(by='num_of_posts_or_comments', ascending=False)

,users,num_of_posts_or_comments,num_antisemetic_comments,max_streanght_antisemitism,sum_antisemitism,mean_strength_antisemitism
18580,JosephL_55,2024,67,10,135,0.066700
1302,pinchasthegris,1384,73,7,133,0.096098
18104,HistorianCertain3758,994,95,8,215,0.216298
7594,avbitran,969,43,7,84,0.086687
16837,JeffB1517,935,49,7,104,0.111230
...,...,...,...,...,...,...
8757,ImJustSomeGuyYouKnow,1,0,0,0,0.000000
15888,Impressive_Toe6080,1,0,0,0,0.000000
15890,Mayaru_san,1,1,7,7,7.000000
15891,ImportantBed9298,1,0,0,0,0.000000


# edges by connection between users

In [318]:
edges_df_A = df_A[['comment_author','submission_author']]
edges_df_A.columns = ['user_A','to_user_B']  
edges_df_B = df_B[['comment_author_x','comment_author_y']]
edges_df_B.columns = ['user_A','to_user_B']
print(f'edges from A {edges_df_A.shape[0]} \nedges from B {edges_df_B.shape[0]}')
edges = pd.concat([edges_df_A, edges_df_B])

edges from A 42518 
edges from B 174157


In [319]:
edegs_with_strength = edges.groupby(['user_A','to_user_B'], as_index=False).size()
edegs_with_strength.head(2)

,user_A,to_user_B,size
0,--Jimmy_Kudo--,Nyeteka,1
1,--Jimmy_Kudo--,coffeemedd,1


In [320]:
with open('edegs_with_strength.pickle', 'wb') as f:
    pickle.dump(edegs_with_strength, f)

In [3]:
with open('edegs_with_strength.pickle', 'rb') as f:
    edegs_with_strength = pickle.load(f)

# extract nodes ( as comments)

In [239]:
import numpy as np

posts = df_posts[df_posts['sum_of_antisemitism'] > 0][['submission_id','vectors','submission_timestamp']]
posts.columns = ['id','vectors','timestamp']

A = df_A[df_A['sum_of_antisemitism'] > 0][['comment_id','vectors','comment_timestamp']]
A.columns = ['id','vectors','timestamp']

B = df_B[df_B['sum_of_antisemitism'] > 0][['comment_id_x','vectors','comment_timestamp']].dropna()
B.columns = ['id','vectors','timestamp']

df_embbeded_stacked = pd.concat([posts,A,B])
df_embbeded_stacked.shape, df_embbeded_stacked['id'].drop_duplicates().shape

((19461, 3), (18650,))

In [189]:
nodes = df_embbeded_stacked[['id','timestamp']]
print(nodes.head(1))

with open('nodes_as_comments.pickle', 'wb') as f:
    pickle.dump(nodes,f)

           id           timestamp
4  t3_16xsk8j 2023-10-02 12:59:31


# edges by content similarity

### calculate distance matrix

#### stack submissions and comments

In [190]:
embedding_stacked = np.stack(df_embbeded_stacked['vectors'].apply(lambda x: np.array(x)).to_numpy())
embedding_stacked.shape

(19461, 768)

### distance via pairwise distance

In [191]:
from sklearn.metrics import pairwise_distances

distance_matrix = pairwise_distances(embedding_stacked)
distance_matrix.shape

(19461, 19461)

#### distance matrix to dataframe

In [192]:
df_distances = pd.DataFrame(distance_matrix, index = df_embbeded_stacked.id, columns = df_embbeded_stacked.id)

In [193]:
df_distances.head(3)

id,t3_16xsk8j,t3_16xz9ex,t3_16zmeyr,t3_170s7w6,t3_1720x34,t3_1724t73,t3_1724v47,t3_1724wch,t3_1725s4a,t3_1726un1,...,t1_k7arigl,t1_k7ariqf,t1_k7arjhf,t1_k7arkx0,t1_k7arztp,t1_k7ashih,t1_k7ashj9,t1_k7asip5,t1_k7aspdt,t1_k7asubb
id,,,,,,,,,,,,,,,,,,,,,
t3_16xsk8j,0.000000,0.724973,0.666123,0.768902,0.945192,0.875764,0.905990,0.887277,0.922448,0.808523,...,0.810145,0.863365,0.753616,0.755882,0.836793,0.796787,0.889288,0.783263,0.820223,0.852142
t3_16xz9ex,0.724973,0.000000,0.759419,0.693710,0.964345,0.899444,0.897542,0.929970,0.908189,0.792150,...,0.874636,0.900126,0.784137,0.770415,0.845984,0.770426,0.855265,0.800440,0.791349,0.903488
t3_16zmeyr,0.666123,0.759419,0.000000,0.799624,0.928363,0.862442,0.873621,0.916755,0.913495,0.797587,...,0.867204,0.899245,0.787935,0.749086,0.871608,0.795774,0.885941,0.767488,0.813679,0.881101


#### distance dataframe to pairs distance (pre-process for edges of the content similarity graph)

In [194]:
pairs = []
for i in range(len(df_distances)):
    for j in range(i + 1, len(df_distances)):
        pairs.append((df_distances.index[i], df_distances.index[j], df_distances.iloc[i, j]))
all_pairs_by_and_distance = pd.DataFrame(pairs, columns=['id1', 'id2', 'distance'])

In [195]:
all_pairs_by_and_distance.shape

(189355530, 3)

In [196]:
all_pairs_by_and_distance.head(3)

,id1,id2,distance
0,t3_16xsk8j,t3_16xz9ex,0.724973
1,t3_16xsk8j,t3_16zmeyr,0.666123
2,t3_16xsk8j,t3_170s7w6,0.768902


#### save point - all pairs with distance

In [197]:
with open('all_pairs_by_and_distance.pickle', 'wb') as f:
    pickle.dump(all_pairs_by_and_distance, f)

In [1]:
import pickle
with open('all_pairs_by_and_distance.pickle', 'rb') as f:
    all_pairs_by_and_distance = pickle.load(f)

#### load all nodes (comments)

In [198]:
with open('nodes_as_comments.pickle', 'rb') as f:
    nodes_as_comments = pickle.load(f)

In [199]:
all_pairs_by_and_distance.shape, nodes_as_comments.shape

((189355530, 3), (19461, 2))

In [200]:
all_pairs_by_and_distance.head(2)

,id1,id2,distance
0,t3_16xsk8j,t3_16xz9ex,0.724973
1,t3_16xsk8j,t3_16zmeyr,0.666123


In [201]:
nodes_as_comments.head(2)

,id,timestamp
4,t3_16xsk8j,2023-10-02 12:59:31
5,t3_16xz9ex,2023-10-02 18:13:24


#### add timestamp to all pairs (for ordering pairs by time)

In [202]:
df_merge1 = all_pairs_by_and_distance.merge(nodes_as_comments, left_on='id1', right_on='id')
df_merge1.columns = ['id1', 'id2', 'distance', 'id', 'timestamp1']
df_merge1.drop(columns='id', inplace=True)

In [203]:
with open('df_merge1.pickle', 'wb') as f:
    pickle.dump(df_merge1, f)

In [ ]:
with open('df_merge1.pickle', 'rb') as f:
    df_merge1 = pickle.load(f)

In [204]:
all_pairs = df_merge1.merge(nodes_as_comments, left_on='id2', right_on='id')
all_pairs.columns = ['id1', 'id2', 'distance', 'timestamp1', 'id', 'timestamp2']
all_pairs.drop(columns='id', inplace=True)

In [206]:
all_pairs.head(2)

,id1,id2,distance,timestamp1,timestamp2
0,t3_16xsk8j,t3_16xz9ex,0.724973,2023-10-02 12:59:31,2023-10-02 18:13:24
1,t3_16xsk8j,t3_16zmeyr,0.666123,2023-10-02 12:59:31,2023-10-04 15:51:13


#### save all pairs with timestamp

In [207]:
with open('all_pairs_by_and_distance_and_dates.pickle', 'wb') as f:
    pickle.dump(all_pairs, f)

In [1]:
import pickle
with open('all_pairs_by_and_distance_and_dates.pickle', 'rb') as f:
    all_pairs = pickle.load(f)

In [2]:
all_pairs.head(2)

,id1,id2,distance,timestamp1,timestamp2
0,t3_16xsk8j,t3_16xz9ex,0.724973,2023-10-02 12:59:31,2023-10-02 18:13:24
1,t3_16xsk8j,t3_16zmeyr,0.666123,2023-10-02 12:59:31,2023-10-04 15:51:13


### calculate clusters based on distance matrix (similar ideas)

In [258]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster

def cluster_from_distance_matrix(distance_matrix: pd.DataFrame, distance_threshold: float):
    """
    Clusters IDs based on the given distance matrix.
    
    Parameters:
        distance_matrix (pd.DataFrame): A symmetric distance matrix where index and columns are the IDs.
        distance_threshold (float): The maximum distance to form a cluster.
        
    Returns:
        dict: Mapping of each ID to its assigned cluster.
    """
    # Ensure the matrix is symmetric
    assert (distance_matrix.values == distance_matrix.values.T).all(), "Distance matrix must be symmetric"

    # Extract the condensed distance matrix (upper triangle, excluding diagonal)
    condensed_distances = distance_matrix.values[np.triu_indices(len(distance_matrix), k=1)]

    # Perform hierarchical clustering
    Z = linkage(condensed_distances, method='single')  # Single linkage for minimum distance clustering

    # Assign cluster labels
    labels = fcluster(Z, distance_threshold, criterion='distance')

    # Map each ID to its cluster
    id_cluster_map = {id_: cluster for id_, cluster in zip(distance_matrix.index, labels)}
    
    cluster_to_id = {}
    for key, val in id_cluster_map.items():
        if val in cluster_to_id:
            cluster_to_id[val].append(key)
        else:
            cluster_to_id[val] = [key]
    
    return cluster_to_id


In [288]:
clusters = cluster_from_distance_matrix(df_distances, distance_threshold=0.4)

#### display clusters sizes

In [289]:
lengths = [len(clusters[key]) for key in clusters.keys()]
set(lengths)

{1, 2, 3, 4, 5, 6, 7, 9, 10, 14, 18, 249}

#### focus on large groups

In [290]:
cluster_size_threshold = 20
[key for key in clusters.keys() if len(clusters[key]) >= 18]
# cluster number 95, are all submissions (need another way to express them in a graph. does not directly lead to users)

[94, 5804]

In [270]:
len(clusters[cluster_number])

249

#### add dates to each comment in the cluster

In [373]:
cluster_number = 94
timestamps_cluster= [df_embbeded_stacked[df_embbeded_stacked['id'] == ids].timestamp.values[0] for ids in clusters[cluster_number]]

In [273]:
marked_edges_df_A = df_A[['comment_id', 'comment_author','submission_author']]
marked_edges_df_A.columns = ['id','user_A','to_user_B']
marked_edges_df_B = df_B[['comment_id_x','comment_author_x','comment_author_y']]
marked_edges_df_B.columns = ['id','user_A','to_user_B']
marked_edges = pd.concat([marked_edges_df_A, marked_edges_df_B])

In [375]:
cluster = {'id':clusters[cluster_number],
             'timestamp' : timestamps_cluster}
pd.DataFrame(cluster)
with open('cluster_5804.pickle', 'wb') as f:
    pickle.dump(marked_edges[marked_edges['id'].isin(clusters[cluster_number])],f)

,id,timestamp
0,t3_1720x34,2023-10-07 11:08:16
1,t3_172bany,2023-10-07 20:06:59
2,t3_172mwp1,2023-10-08 04:39:54
3,t3_172q42n,2023-10-08 07:28:24
4,t3_172u1uo,2023-10-08 11:31:39
...,...,...
244,t3_17k3okk,2023-10-30 23:05:00
245,t3_17kgsf4,2023-10-31 11:11:38
246,t3_17kpu8c,2023-10-31 19:03:26
247,t3_17kpva0,2023-10-31 19:04:45


#### check if texts are submissions (starts with "t3") or comments (starts with "t1")

In [95]:
pd.DataFrame(cluster_95).id.str.startswith('t3')

0      True
1      True
2      True
3      True
4      True
       ... 
244    True
245    True
246    True
247    True
248    True
Name: id, Length: 249, dtype: bool

#### display all comments of cluster 5804

In [371]:
marked_edges[marked_edges['id'].isin(clusters[cluster_number])]
# pd.DataFrame(cluster)

,id,user_A,to_user_B
15243,t1_k503olf,voyagercw1996,AchPzYlahyklk
38596,t1_k70kc8r,AmnesiacPatient,Goobee69
38598,t1_k70kddz,AmnesiacPatient,ronthegr8
38600,t1_k70ke7g,AmnesiacPatient,orgad
38601,t1_k70kft7,AmnesiacPatient,Additional_Novel_944
38602,t1_k70kgpr,AmnesiacPatient,Morgentau7
38603,t1_k70kice,AmnesiacPatient,gsitk
38605,t1_k70kj7o,AmnesiacPatient,avbitran
38606,t1_k70kk46,AmnesiacPatient,mikebenb
38607,t1_k70koro,AmnesiacPatient,mikebenb


In [378]:
ids = 't3_17kgsf4'
get_text_by_id(ids)

array(['[removed]'], dtype=object)

In [362]:
df_B.columns

Index(['submission_id', 'submission_title', 'submission_text', 'comment_id_x',
       'comment_text_x', 'comment_author_x', 'to_comment', 'comment_text_y',
       'comment_author_y', 'comment_timestamp', 'len_submission_text',
       'len_comment_text_x', 'len_comment_text_y', 'index', 'features',
       'promote or endorse harmful stereotypes about Jewish people (e.g., controlling the media, financial systems, or governments)?',
       'blame Jewish people collectively for societal problems, political events, or economic issues?',
       'deny, minimize, or distort the Holocaust or other historical antisemitic events?',
       'promote conspiracy theories that falsely portray Jewish people as a secretive, manipulative, or powerful group controlling world affairs?',
       'use derogatory language, slurs, or dehumanizing descriptions when referring to Jewish people?',
       'accuse Jewish people of exaggerated or false crimes, such as blood libel?',
       'compare Jewish people or Is

In [359]:
df_A[df_A['comment_id']=='t1_k4t19xo'].comment_text

Series([], Name: comment_text, dtype: object)

In [369]:
df_B.columns
df_A[df_A['comment_author']=='Afraid-Carry4093'].comment_text.iloc[1]

'Thank you for your perspective. \n\nJust like you, their is a 14  year old Palestenian kid in Gaza strip feeling the same exact way. Their are many antrocities that both the IDF and Hamas have done to each side for years. Some more barbaric than others. This attack being one of them.\n\n Also, i know that IDF horrific actions, does not represent the Israeli people. Just like Hamas terrorists horrific actions does not represent the whole Palestinians. Their is a reason this attack is so controversial and many not feeling sorry for Israel. Its not black and white like the Russian invasion of Ukraine. The conflict between Israel and Gaza is so much more complicated.\n\n\nHaving said that, their is no justification on Hamas attack on Israeli civilians. The kidnappings, torture, rape, murder and parading of dead bodies of elderly, women and kids is horric and the most barbaric unforgivable things done. \n\nIt is gross seeing palestian immigrants celebrating in western countries of this att

### insert timestamp aspect to edges  

In [342]:
edges_df_A = df_A[['comment_author','submission_author','comment_timestamp']]
edges_df_A.columns = ['user_A','to_user_B','timestamp']  
edges_df_B = df_B[['comment_author_x','comment_author_y','comment_timestamp']]
edges_df_B.columns = ['user_A','to_user_B','timestamp']
print(f'edges from A {edges_df_A.shape[0]} \nedges from B {edges_df_B.shape[0]}')
edges = pd.concat([edges_df_A, edges_df_B])

edges from A 42518 
edges from B 174157


In [347]:
with open('edges_with_timestamp.pickle', 'wb') as f:
    pickle.dump(edges.sort_values(by='timestamp').groupby(['user_A','to_user_B'], as_index=False).first(), f)